In [1]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer, Normalizer
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice

from annsa.template_sampling import *

In [2]:
from hyperparameter_models import make_conv1d_model as make_model

In [3]:
import tensorflow.contrib.eager as tfe

In [4]:
tf.enable_eager_execution()

#### Import model, training function 

In [5]:
from annsa.model_classes import (cnn1d_model_features,
                                 CNN1D,
                                 generate_random_cnn1d_architecture,
                                 save_model,
                                 train_earlystop)

## Load testing dataset 

In [6]:
dataset = np.load('../dataset_generation/testing_dataset_200keV_100.npy')

In [7]:
all_spectra = np.add(dataset.item()['sources'], dataset.item()['backgrounds'])
all_keys = dataset.item()['keys']

mlb=LabelBinarizer()

all_keys_binarized = mlb.fit_transform(all_keys)

## Train network

### Define hyperparameters

In [8]:
number_hyperparameters_to_search = 256
earlystop_errors_test = []
model_id='CNN-kfoldseasy-updated-smaller-5'

### Search hyperparameters

In [ ]:
def save_features(model_features,
                  model_id,
                  hyperparameter_index):
    with open('./hyperparameter-search-results/' + model_id + '_' +
              str(hyperparameter_index), 'wb+') as f:
        pickle.dump(model_features,f)
    return None

skf = StratifiedKFold(n_splits=5, random_state=5)
testing_errors = []
all_kf_errors = []

for network_id in range(number_hyperparameters_to_search):
    print(network_id)
    model, model_features = make_model(all_keys_binarized)
    save_features(model_features,
                  model_id,
                  network_id)
    
    k_folds_errors = []
    for train_index, test_index in skf.split(all_spectra, all_keys):
        # reset model on each iteration
        model = CNN1D(model_features)
        optimizer = tf.train.AdamOptimizer(model_features.learining_rate)

        costfunction_errors_tmp, earlystop_errors_tmp = train_earlystop(
                training_data=all_spectra[train_index],
                training_keys=all_keys_binarized[train_index],
                testing_data=all_spectra[test_index],
                testing_keys=all_keys_binarized[test_index],
                model=model,
                optimizer=optimizer,
                num_epochs=200,
                obj_cost=model.cross_entropy,
                earlystop_cost_fn=model.f1_error,
                earlystop_patience=10,
                not_learning_patience=10,
                not_learning_threshold=0.9,
                verbose=True,
                fit_batch_verbose=10,
                data_augmentation=model.default_data_augmentation)
        k_folds_errors.append(earlystop_errors_tmp)
        all_kf_errors.append(earlystop_errors_tmp)

    testing_errors.append(np.average(k_folds_errors))
    np.save('./final-models/final_test_errors_'+model_id, testing_errors)
    np.save('./final-models/final_kf_errors_'+model_id, all_kf_errors)
    # model.save_weights('./final-models/'+model_id+'_checkpoint_'+str(network_id))
    network_id += 1 

0
Epoch 10: CostFunc loss: 0.00 3.44, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.47 Early stopfctn: 3.47
Epoch 10: CostFunc loss: 0.00 3.47, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.49 Early stopfctn: 3.49
Epoch 10: CostFunc loss: 0.00 3.46, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.49 Early stopfctn: 3.49
Epoch 10: CostFunc loss: 0.00 3.46, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.48 Early stopfctn: 3.48
Epoch 10: CostFunc loss: 0.00 3.45, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.48 Early stopfctn: 3.48
1
Epoch 10: CostFunc loss: 0.00 2.51, EarlyStop loss: 0.00 0.76
Epoch 20: CostFunc loss: 0.00 1.96, EarlyStop loss: 0.00 0.61
Epoch 30: CostFunc loss: 0.00 1.66, EarlyStop loss: 0.00 0.52
Epoch 40: CostFunc loss: 0.00 1.42, EarlyStop loss: 0.00 0.47
Epoch 50: CostFunc loss: 0.00 1.20, EarlyStop loss: 0.00 0.39
Epoch 60: CostFunc loss: 0.00 1

Test error at early stop: Objectives fctn: 3.45 Early stopfctn: 3.45
Epoch 10: CostFunc loss: 0.00 3.43, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.45 Early stopfctn: 3.45
Epoch 10: CostFunc loss: 0.00 3.44, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.45 Early stopfctn: 3.45
Epoch 10: CostFunc loss: 0.00 3.43, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.44 Early stopfctn: 3.44
7
Epoch 10: CostFunc loss: 0.00 3.68, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.76 Early stopfctn: 3.76
Epoch 10: CostFunc loss: 0.00 3.70, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.78 Early stopfctn: 3.78
Epoch 10: CostFunc loss: 0.00 3.73, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.82 Early stopfctn: 3.82
Epoch 10: CostFunc loss: 0.00 3.65, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.70 Early stopfctn: 3.70
Epoch 10: Co